# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание используемух столбцов:**
* children — количество детей в семье  
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

**Краткий план работы:**
1. Открытие таблицы и изучение общей информации о данных
2. Предобработка данных
3. Ответы на поставленные в проекте вопросы
4. Общие выводы

## Шаг 1. Откройте файл с данными и изучите общую информацию

Выведем первые 5 строк данных

In [1]:
import pandas as pd
from IPython.display import display
solvency_statistics = pd.read_csv('/datasets/data.csv')
display(solvency_statistics.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Посмотрим на последние 5 строк

In [2]:
display(solvency_statistics.tail(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Взглянем на информацию о данных

In [3]:
solvency_statistics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Входные данные имеют следующие проблемы:
1. Есть пропущенные значения в категориях: days_employed, total_income;
2. Некорректное отображение значений в разделе days_employed: трудовой стаж не может быть отрицательным;
3. В категории education есть дубликаты в разном регистре; 
4. Колонки days_employed и total_income нужно привести к читаемому виду, методом замены типа данных;
5. В колонке purpose есть дубли с одинаковым смыслом, но по разному описанные.

## Шаг 2. Предобработка данных

### Обработка пропусков

Найдём все строки с пропусками общего трудового стажа и посмотрим на первые 5

In [4]:
display(solvency_statistics[solvency_statistics['days_employed'].isnull()].head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Найдём все строки с пропусками ежемесячного дохода и посмотрим на первые 5: строки с отсутствующим СТАЖЕМ совпадают со строками с отсутствующим ДОХОДОМ.

In [5]:
solvency_statistics[solvency_statistics['total_income'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [6]:
solvency_statistics[solvency_statistics['days_employed'].isnull()].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Посчитали количество строк с отсутствующими данными. Их 2174.

Заменим отсутствующие значения на 0.

In [7]:
solvency_statistics[['days_employed', 'total_income']] = solvency_statistics[['days_employed', 'total_income']].fillna(value=0)

Выведем статистику ещё раз: нулевые значения отсутствуют.

In [8]:
solvency_statistics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

1. Заменить пустые ячейки на какие-либо реальные значения мы не можем. Так как не прослеживается закономерность с известными данными.
2. Удалить строки с пустыми значениями мы так же не можем. Их большое количество (2174) и удаление повлияет на выявление зависимости по другим известным параметрам (количество детей, семейное положение и т.д.)
3. Возможная причина отсутствия данных по СТАЖУ и ДОХОДУ это маленькая сумма кредита, которая выдаётся даже без подтверждения дохода и наличия места работы.
4. Пусты значения были заменены на 0, для возможности дальнейших преобразований таблицы. Для расчёта зависимости от уровня дохода строки с нулями нужно будет исключить.

### Замена типа данных

Заменим тип данных в столбцах СТАЖА и ДОХОДА с вещественного на целочисленный

In [9]:
solvency_statistics[['days_employed', 'total_income']] = solvency_statistics[['days_employed', 'total_income']].astype('int')
solvency_statistics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Уберём минусы в столбце days_employed. Стаж работы не может быть отрицательным.  
Уберём минусы в столбце children. Количество детей не может быть отрицательным.

In [10]:
solvency_statistics[['days_employed', 'children']] = solvency_statistics[['days_employed', 'children']].abs()
solvency_statistics.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Для изменения типа данных в столбцах СТАЖА и ДОХОДА, был выбран метод astype(), так как нам нужно получить целочисленные значения из вещественных.

### Обработка дубликатов

Приведём к нижнему регистру значения в столбце ОБРАЗОВАНИЕ

In [11]:
solvency_statistics['education'] = solvency_statistics['education'].str.lower() 
solvency_statistics.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [12]:
solvency_statistics.duplicated().sum()

71

Проверили количество полных дубликатов строк. Получилось 71.

In [13]:
solvency_statistics = solvency_statistics.drop_duplicates().reset_index(drop = True)
solvency_statistics.duplicated().sum()

0

Применили метод drop_duplicates() для удаления дубликатов и проверили количество дублей снова. Получилось 0. Ура.

**Вывод**

Мы обнаружили и удалили 71 строку полных дубдей. Дубликаты могли появится в таблице по причине некоректной выгрузки данных из другой информационной системы.

### Лемматизация

Посмотрим на уникальные элементы столбца purpose

In [14]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem() 
purpose_unique = solvency_statistics['purpose'].unique()
purpose_unique

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Отсортируем по убыванию все лемматизированные слова

In [15]:
purpose_unique = ' '.join(purpose_unique)
purpose_words = m.lemmatize(purpose_unique)
Counter(purpose_words).most_common()

[(' ', 96),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('подержать', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('со', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1),
 ('\n', 1)]

Выделим основные цели получения кредита, это: недвижимость, автомобиль, образование, жильё и свадьба.

Создадим новый столбец, в котором выделим леммы с целями получения кредита.

In [16]:
def purpose_lemmas(data):
    words = m.lemmatize(data['purpose'])
    for word in words:
        if 'жилье' in words or 'недвижимость' in words:
            return 'недвижимость'
        if 'автомобиль' in words:
            return 'автомобиль'
        if 'образование' in words:
            return 'образование'
        if 'свадьба' in words:
            return 'свадьба'
        
solvency_statistics['purpose_lemmas'] = solvency_statistics.apply(purpose_lemmas, axis = 1)

Взглянем на новую таблицу

In [17]:
solvency_statistics.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


Проверим, что охватили все цели получения кредита. 0 пустых значений: значит всё получилось.

In [18]:
solvency_statistics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
purpose_lemmas      21454 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


**Вывод**

В данном разделе мы выделили основные цели получения кредита, это: недвижимость, автомобиль, образование, жильё и свадьба.
Создали новый столбец с леммами целей получения кредита, для удобства оценки влияния целей на возврат кредита.

### Категоризация данных

Посмотрим статистику по количеству детей

In [19]:
solvency_statistics['children'].value_counts()

0     14091
1      4855
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Скорее всего 20 детей это опечатка, заменим данное значение на 2.  
Проверим корректность данных.

In [20]:
solvency_statistics['children'] = solvency_statistics['children'].replace(20, 2)
solvency_statistics['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Рассчитаем долю не корректных значений, с числом детей равным 20.

In [21]:
children_20 = 76/21454
print('{:.2%}'.format(children_20))

0.35%


Добавим новый столбец для категоризации по количеству детей

In [22]:
def children_category(data):
    if data['children'] == 0:
        return 'бездетные'
    elif 1 <= data['children'] <= 2:
        return '1 или 2 ребенка'
    else:
        return 'многодетные'
    
solvency_statistics['children_category'] = solvency_statistics.apply(children_category, axis=1)

In [23]:
solvency_statistics.groupby(['family_status', 'family_status_id']).count()

,,children,days_employed,dob_years,education,education_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,children_category
family_status,family_status_id,,,,,,,,,,,,
Не женат / не замужем,4,2810,2810,2810,2810,2810,2810,2810,2810,2810,2810,2810,2810
в разводе,3,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195
вдовец / вдова,2,959,959,959,959,959,959,959,959,959,959,959,959
гражданский брак,1,4151,4151,4151,4151,4151,4151,4151,4151,4151,4151,4151,4151
женат / замужем,0,12339,12339,12339,12339,12339,12339,12339,12339,12339,12339,12339,12339


Посмотрим статистику по семейному положению: неточностей не выявленно.

Узнаем максимальное и минимальное значение  ежемесячного дохода:

In [24]:
print(solvency_statistics['total_income'].max())
print(solvency_statistics['total_income'].min())

2265604
0


Добавим новый столбец для категоризации по доходу

In [25]:
def income_category(data):
    if data['total_income'] <= 50000:
        return 'низкий доход'
    elif 50000 < data['total_income'] <= 150000:
        return 'средний доход'
    elif 150000 < data['total_income'] < 1000000:
        return 'высокий доход'
    else:
        return 'огромный доход'
    
solvency_statistics['income_category'] = solvency_statistics.apply(income_category, axis=1)

In [26]:
display(solvency_statistics.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,children_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,1 или 2 ребенка,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1 или 2 ребенка,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,бездетные,средний доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетные,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,бездетные,высокий доход


**Вывод**

Мы произвели категоризацию данных: по количеству детей, по семейному статусу и по уровню ежемесячного дохода. Добавили новые столбцы в таблице для наглядности.
Можно переходить к анализу данных.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [27]:
def debt_func(category):
    debt_frame = solvency_statistics.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')
    return debt_frame

debt_func('children_category')

,debt
children_category,
бездетные,0.075438
многодетные,0.081579
1 или 2 ребенка,0.092654


**Вывод**

Бездетные заемщики чаще возвращают кредит в срок. 
Заемщики с 1-им или 2-мя детьми более склоны к просрочке по кредиту. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [28]:
debt_func('family_status')

,debt
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075452
гражданский брак,0.093471
Не женат / не замужем,0.097509


**Вывод**

Заемщики не состоящие в браке более склоны к просрочке по кредиту, чем женатые (замужние). 


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [29]:
debt_func('income_category')

,debt
income_category,
низкий доход,0.077980
высокий доход,0.079048
огромный доход,0.080000
средний доход,0.083920


**Вывод**

Всех ответственней к выплате кредита в срок подходят заемщики с доходом менее 50 тыс.руб. в месяц.
Сумые безответственные заемщики со средним доходом от 50 до 150 тыс. руб.

- Как разные цели кредита влияют на его возврат в срок?

In [30]:
debt_func('purpose_lemmas')

,debt
purpose_lemmas,
недвижимость,0.072334
свадьба,0.080034
образование,0.092200
автомобиль,0.093590


**Вывод**

Люди берущие кредит для приобретения жилья более ответсвенно подходят к вопросу выплаты кредита в срок. Всех больше просрочек у заемщиков целью покупки которых является автомобиль.

## Шаг 4. Общий вывод

Самыми ответственными заемщиками являются люди: не имеющие детей, находящиеся в официальном браке (или ранее были в таковом), с небольшим уровнем дохода (менее 50 тыс.), целью кредита которых является недвижимость.

Самыми безответственными заемщиками являются люди: имеющие 1ого (или 2х) детей, не состоящие в браке (или состоящие в гражданском браке), со средним (от 50 до 150 тыс.) уровнем дохода, целью кредита которых является автомобиль.